<a href="https://colab.research.google.com/github/CrashingGuru/FGAN-Build-a-thon/blob/main/Notebooks2023/Argilla_Supervised_Finetuning-v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Created: 2 Jan 2024  

Aaron, Othniel, Kennedy, Vishnu.

Modification History: 4 Jan 2024: Aaron, Frank, Othniel, Vishnu: Changed the data schema to a simpler format. Bye-bye "for_supervised_fine_tuning" format.
Description:

This notebook pulls the dataset from remote argilla and pushes it to the HF hub.
Prerequisites:

the following notebooks are already run:

0. Create the raw dataset in HF hub.

1. Manually create a HF spaces deployment of Argilla

2. Configure the argilla dataset

2. add records in the argilla dataset from the raw dataset

3. ##annotate the dataset in UI

and finally here we do -- Pull from argilla and Push to HF hub to annotated dataset.


## Install Libraries

Install the latest version of Argilla in Colab, along with other libraries and models used in this notebook.

In [ ]:
!pip install argilla datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00


Prerequisites

Deploy Argilla Server on [HF Spaces](https://huggingface.co/new-space?template=argilla/argilla-template-space).


More info on Installation [here](../getting_started/installation/deployments/deployments.html).

## Secretes needed




* `ARGILLA_API_URL`: It is the url of the Argilla Server.
  * If you're using HF Spaces, it is constructed as `https://[your-owner-name]-[your_space_name].hf.space`.
* `ARGILLA_API_KEY`: It is the API key of the Argilla Server. It is `owner` by default.
* `HF_TOKEN`: It is the Hugging Face API token. It is only needed if you're using a [private HF Space](https://docs.argilla.io/en/latest/getting_started/installation/deployments/huggingface-spaces.html#deploy-argilla-on-spaces). You can configure it in your profile: [Setting > Access Tokens](https://huggingface.co/settings/tokens).
* `workspace`: admin


In [ ]:
import argilla as rg
from argilla._constants import DEFAULT_API_KEY

In [ ]:
from google.colab import userdata
api_url= userdata.get('my_argilla_url')
api_key= userdata.get('my_argilla_key')

import argilla as rg
rg.init(api_url=api_url, api_key=api_key)




# # If you want to use your private HF Space
# rg.init(extra_headers={"Authorization": f"Bearer {hf_token}"})

/usr/local/lib/python3.10/dist-packages/argilla/client/client.py:168: UserWarning: No workspace configuration was detected. To work with Argilla datasets, specify a valid workspace name on `rg.init` or set it up through the `rg.set_workspace` function.
  warnings.warn(


In [ ]:
hf_token = userdata.get('my_hf_write_token')
from huggingface_hub import login
login(hf_token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
#refer: https://docs.argilla.io/en/latest/practical_guides/collect_responses.html
#"To collect the responses given by annotators via Python, you can simply load the dataset from Argilla"
#so we collect the responses from the dataset deployed
#for annotation with argilla, deployed in HF spaces
#(and not the raw dataset in the HF hub)
dataset = rg.FeedbackDataset.from_argilla(name="fgan-annotate-dataset", workspace="admin")


In [ ]:
# Push to HuggingFace Hub
dataset.push_to_huggingface("vishnuramov/itu_annotated_dataset")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/9.63k [00:00<?, ?B/s]

Go to HF hub datasets
and cross check if the dataset reflects the annotations.
(and have fun)

Remember to reload the dataset page or you are bound to be disappointed.

-------------

